In [1]:
import os

In [2]:
%pwd

'c:\\Users\\saatv\\Documents\\Programming\\Text-Summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\saatv\\Documents\\Programming\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# defining entity
@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
# defining configuration
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\saatv\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-21 15:30:13,111: INFO: config: PyTorch version 2.3.0 available.]


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model= model_pegasus)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        #reading from yaml file
        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs = self.config.num_train_epochs,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_train_batch_size,
            save_steps = float(self.config.save_steps),
            eval_strategy = self.config.eval_strategy,
            eval_steps = self.config.eval_steps,
            logging_steps = self.config.logging_steps,
            warmup_steps = self.config.warmup_steps,
            weight_decay = self.config.weight_decay,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps
        )

        # #inputting own args
        # trainer_args = TrainingArguments(
        #     output_dir = self.config.root_dir,
        #     num_train_epochs = 1,
        #     per_device_train_batch_size = 1,
        #     per_device_eval_batch_size = 1,
        #     save_steps = float(1e6),
        #     eval_strategy = 'steps',
        #     eval_steps = 500,
        #     logging_steps = 10,
        #     warmup_steps = 500,
        #     weight_decay = 0.01,
        #     gradient_accumulation_steps = 16
        # )

        trainer = Trainer(model = model_pegasus, args = trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                            train_dataset = dataset_samsum_pt['test'],
                            eval_dataset = dataset_samsum_pt['validation'])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-tokenizer"))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-21 15:30:13,741: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-21 15:30:13,756: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-21 15:30:13,759: INFO: common: Directory created at: artifacts]
[2024-05-21 15:30:13,761: INFO: common: Directory created at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 20%|█▉        | 10/51 [1:08:46<4:20:10, 380.75s/it]

{'loss': 3.1101, 'grad_norm': 31.952699661254883, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [2:04:49<2:39:56, 309.56s/it]

{'loss': 3.0466, 'grad_norm': 15.817596435546875, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.9901, 'grad_norm': 20.358823776245117, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [7:04:23<15:12, 912.50s/it]   

{'loss': 2.8543, 'grad_norm': 40.65480422973633, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [7:09:12<00:00, 504.95s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'train_runtime': 25752.2916, 'train_samples_per_second': 0.032, 'train_steps_per_second': 0.002, 'train_loss': 3.023241314233518, 'epoch': 1.0}
